In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from models.slm import ScalableLM
from models.config import LlamaCLConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
dataset = load_dataset("Stevross/mmlu", "high_school_european_history", split="auxiliary_train")
# model = ScalableLM(LlamaCLConfig())
# model.load_state_dict(torch.load("/home/user/bhpeng/SLM-llama/outputs/history/history/checkpoint-780/pytorch_model_retriever.bin"), strict=False)

/data/miniconda3/envs/llm_seg/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-17 15:31:27,770] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
import re
PROMPT_TEMPLATE = {
'with_sys': """[INST] <<SYS>>
{instruction}
<</SYS>>

{input} [/INST] \n""",

'without_sys': """[INST] {input} [/INST] \n"""
}
def extract_from_history(example):
    question = example['question']
    choices = example['choices']
    output = example['answer']
    input="Question:{}\n Choices:{}\n".format(question, str(choices))
    output = chr(ord('@')+int(output)+1)
    return dict(
        input=input,
        output=output
    )

def prepare_data(example):
    example = extract_from_history(example)
    raw_text = example['instruction'] + " ### " + example['input'] if 'instruction' in example.keys() else \
                   example['input']
    source = PROMPT_TEMPLATE['with_sys'].format_map(example) if 'instruction' in example.keys() else\
                 PROMPT_TEMPLATE['without_sys'].format_map(example)
    source = tokenizer(
            source,
            max_length=512,
            truncation=True,
            return_tensors='pt'
        )
    task = 'history'
    return dict(
        raw_text=raw_text,
        input_ids=source['input_ids'],
        attention_mask=source['attention_mask'],
        task=task
    ), example['output'] + "</s>"

In [28]:
import random
inps, outs = prepare_data(dataset[random.randint(0, 1000)])
# out = model.generate(**inps)
print(outs)
# print(tokenizer.decode(out[0]))
print(inps['raw_text'])

A</s>
Question:Dean, charged with murder, was present with her attorney at a preliminary examination when White, who was the defendant in a separate prosecution for concealing the body of the murder victim, testified for the prosecution against Dean. When called to testify at Dean's trial, White refused to testify, though ordered to do so. The prosecution offers evidence of White's testimony at the preliminary examination. The evidence is
 Choices:['admissible as former testimony.', 'admissible as past recollection recorded.', "inadmissible, because it would violate White's privilege against selfincrimination. ", 'inadmissible, because it is hearsay, not within any exception']

